In [1]:
from pprint import pprint as print
import requests
import json
import time
from IPython.display import clear_output
from pprint import pprint as print

APP_URL = "http://jsl-resolvers-api:5000"
HEADERS = {
  'Content-Type': 'application/json'
}

RESOLVERS_URL = f"{APP_URL}/resolvers"
STATUS_URL = f"{APP_URL}/status"
DEPLOY_URL = f"{APP_URL}/deploy"
SEARCH_URL = f"{APP_URL}/search"
RESET_URL = f"{APP_URL}/reset"

## Status and Resolvers

Shows the available resolver list.

In [2]:
requests.get(RESOLVERS_URL).json()

{'jsl_resolvers': {'ATC': '142.36 MB',
  'HCC': '2.61 GB',
  'HCPCS': '37.64 MB',
  'HGNC': '506.47 MB',
  'ICD10CM': '2.62 GB',
  'ICD10PCS': '1.27 GB',
  'ICDO': '356.48 MB',
  'MESH': '2.47 GB',
  'NCIT': '1.93 GB',
  'NDC': '1.44 GB',
  'RXNORM': '2.23 GB',
  'SNOMED': '4.27 GB'},
 'current_server_info': {'deployment_status': [{'ICD10CM': {'deployed_at': '2024-09-23 19:26:10 UTC',
     'device': 'cpu'}},
   {'HCPCS': {'deployed_at': '2024-09-23 19:33:11 UTC', 'device': 'cpu'}}],
  'ready': 'ICD10CM, HCPCS'}}

In [3]:
# check server status

requests.get(STATUS_URL).json()

{'server': 'up',
 'current_server_info': {'deployment_status': [{'ICD10CM': {'deployed_at': '2024-09-23 19:26:10 UTC',
     'device': 'cpu'}},
   {'HCPCS': {'deployed_at': '2024-09-23 19:33:11 UTC', 'device': 'cpu'}}],
  'ready': 'ICD10CM, HCPCS'}}

## Deployment

Deploy the resolver you want to use.

In [4]:
RESOLVER_NAME = "ICD10CM"
payload = {"name": RESOLVER_NAME}
requests.post(DEPLOY_URL, headers=HEADERS, data=json.dumps(payload)).json()

{'message': 'starting'}

### Check status of deployment

In [6]:
def track_deployment_status():
    while True:
        if deployment_status := requests.get(STATUS_URL).json().get("current_server_info", {}).get("deployment_status"):
            clear_output(wait=True)
            print(deployment_status)
            in_progress = False
            for each in deployment_status:
                if each.get("progress"):
                    in_progress = True
            if not in_progress:
                break
            time.sleep(10)
        else:
            break


track_deployment_status()

[{'ICD10CM': {'deployed_at': '2024-09-24 13:50:18 UTC', 'device': 'cpu'}},
 {'HCPCS': {'deployed_at': '2024-09-23 19:33:11 UTC', 'device': 'cpu'}}]


## Search

Get the results of the queries from the resolver.

In [7]:
# for list of chunks per document

payload = {
        "name": RESOLVER_NAME,
        "docid_entities": {
            "doc_1": ["lung cancer", "brain tumor", "lungx cancer"],
            "doc_2": ["Alzheimer's disease", "stroke", "breast cancer"],
        },
        "topk": 2
    }

requests.post(SEARCH_URL, headers=HEADERS, data=json.dumps(payload)).json()

{'results': {'doc_1': [{'lung cancer': [{'resolved_text': 'lung cancer',
      'billable_hcc': '1||1||9',
      'concept_code': 'C34.90',
      'concept_name_detailed': 'lung cancer [malignant neoplasm of unspecified part of unspecified bronchus or lung]',
      'score': 1.0},
     {'resolved_text': 'carcinoma of lung',
      'billable_hcc': '0||0||0',
      'concept_code': 'C34.9',
      'concept_name_detailed': 'carcinoma of lung [malignant neoplasm of unspecified part of bronchus or lung]',
      'score': 0.8682}]},
   {'brain tumor': [{'resolved_text': 'brain tumor',
      'billable_hcc': '1||1||12',
      'concept_code': 'D49.6',
      'concept_name_detailed': 'brain tumor [neoplasm of unspecified behavior of brain]',
      'score': 1.0},
     {'resolved_text': 'brain tumour',
      'billable_hcc': '1||1||12',
      'concept_code': 'D43.2',
      'concept_name_detailed': 'brain tumour [neoplasm of uncertain behavior of brain, unspecified]',
      'score': 0.9374}]},
   {'lungx can

## Reset

In [7]:
requests.post(RESET_URL).json()

{'message': 'reset done'}